In [1]:
import requests
import lxml.html as lh
import pandas as pd

## Récupération des données par web Scraping

Récupération d'une liste de jeu simple avec nom, desription et date de sortie : http://www.jeuxvideo.com/meilleurs/machine-20/

In [2]:
url='http://www.jeuxvideo.com/meilleurs/machine-20/'
page = requests.get(url)
doc = lh.fromstring(page.content)

On récupère les données via des requêtes XPath

In [3]:
nomJeux = doc.xpath('//a[@class="gameTitleLink__196nPy"]')
desc = doc.xpath('//p[@class="description__1-Pqha"]')
sortie = doc.xpath('//span[@class="releaseDate__1RvUmc"]')

Affichage des données décomposées :

In [64]:
for i in range(len(nomJeux)):
    print(nomJeux[i].text_content().replace(" sur PS4", "") + " \n" + \
          desc[i].text_content() + " \n" + \
          sortie[i].text_content()+ " \n")

God of War 
Dans ce nouvel épisode de God Of War, le héros évoluera dans un monde aux inspirations nordiques, très forestier et montagneux. Dans ce beat-them-all, un enfant accompagnera le héros principal, pouvant apprendre des actions du joueur, et même gagner de l'expérience. 
Sortie: 20 avril 2018 

Divinity : Original Sin II - Definitive Edition 
Divinity : Original Sin II sur PS4 est un RPG où vous choisissez votre race et et votre background. Le monde réagit en fonction de vos choix. Rassemblez votre équipe et affrontez vos adversaires avec différents éléments de combat tactique au tour par tour. Explorez un vaste monde seul ou avec 4 joueurs en coopératif. 
Sortie: 31 août 2018 

The Witcher 3 : Wild Hunt 
Action-RPG en open world pour PS4, The Witcher 3 : Wild Hunt est le troisième opus de la série de jeux éponyme. Le joueur y retrouve le personnage de Geralt de Riv pour découvrir la fin de son histoire mouvementée. 
Sortie: 19 mai 2015 

The Last of Us Remastered 
The Last of 

### Mettons les données dans un DataFrame Pandas

In [66]:
l1 = pd.DataFrame([nomJeux[i].text_content().replace(" sur PS4", "") for i in range(len(nomJeux))], columns=['nomJeux'])
l1['key'] = l1.index
l2 = pd.DataFrame([desc[i].text_content() for i in range(len(desc))], columns=['Description'])
l2['key'] = l2.index
l3 = pd.DataFrame([sortie[i].text_content().replace("Sortie: ", "") for i in range(len(sortie))], columns=['Sortie'])
l3['key'] = l3.index
liste = l1.join(l2, on='key', how='left', lsuffix='_l1', rsuffix='_r1').join(l3, on='key_l1', how='left', lsuffix='_l1', rsuffix='_r2')
del liste['key_l1']
del liste['key_r1']
liste

,nomJeux,Description,Sortie,key
0,God of War,"Dans ce nouvel épisode de God Of War, le héros...",20 avril 2018,0
1,Divinity : Original Sin II - Definitive Edition,Divinity : Original Sin II sur PS4 est un RPG ...,31 août 2018,1
2,The Witcher 3 : Wild Hunt,"Action-RPG en open world pour PS4, The Witcher...",19 mai 2015,2
3,The Last of Us Remastered,The Last of Us Remastered est un survival acti...,30 juillet 2014,3
4,Inside,Inside est un j'au d'aventure et d'exploration...,23 août 2016,4
5,Uncharted 4 : A Thief's End,Quatrième opus de la série de jeu d'action/ave...,10 mai 2016,5
6,Day of the Tentacle : Special Edition,Day of the Tentacle : Special Edition est un j...,22 mars 2016,6
7,The Walking Dead : Saison 1,The Walking Dead sur PS4 est l'adaptation du c...,24 octobre 2014,7
8,The Walking Dead : Episode 5 - No Time Left,The Walking Dead : Episode 5 - No Time Left po...,14 octobre 2014,8
9,Divinity : Original Sin - Enhanced Edition,"Jeu de rôle au tour par tour, Divinity : Origi...",27 octobre 2015,9
